In [1]:
import os 

In [2]:
os.chdir('../')

In [6]:
pwd

'c:\\Users\\HP\\Chicken-Disease-Classification-Raj43'

In [7]:
import tensorflow as tf


ModuleNotFoundError: No module named 'tensorflow'

In [8]:
model=tf.keras.models.load_model('artifacts/training/model.h5')

NameError: name 'tf' is not defined

In [5]:
from dataclasses import dataclass 
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model:Path
    training_data:Path
    all_params:dict
    params_image_sizeL:list
    params_batch_size:int


In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utlis import read_yaml,create_directories,save_json

class ConfigurationManager:
    def __init__(
            self,
            config_filepath=CONFIG_FILE_PATH,
            params_filepath=PARAMS_FILE_PATH):
            
            self.config=read_yaml(config_filepath)
            self.params=read_yaml(params_filepath)
            create_directories(self.config.artifacts_root)
    def get_validation_config(self)->EvaluationConfig:
          eval_config=EvaluationConfig(
                path_of_model="artifacts/training/model.h5",
                training_data="artifacts/data_ingestion/Chiken-fecal-images",
                all_params=self.params,
                params_image_size=self.params.IMAGE_SIZE,
                params_batch_size=self.params.BATCH_SIZE
          )

          return eval_config
    
           
    
    

ModuleNotFoundError: No module named 'cnnClassifier'

In [13]:
from urllib.parse import urlparse

class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config=config
    
    def _valid_generator(self):

        datagenerator_kwargs=dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs=dict(
            target_size=self.config.params_image_sizeL[:-1],
            interpoation='bilinear'
        )

        valid_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )
    @staticmethod
    def load_model(path,Path)->tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model=self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score=model.evaluate(self.valild_generator)

    def save_score(self):
        scores={"loss":self.score[0],"accuracy":self.score[1]}
        save_json(path=Path('score.json'),data=scores)


NameError: name 'tf' is not defined

In [ ]:

try:
    config=ConfigurationManager()
    val_config=config.get_validation_config()
    evaluation=Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e
